In [1]:
%load_ext autoreload 
%autoreload 2
%matplotlib notebook
import matplotlib.pyplot as plt
import random
import numpy as np
import sys
import os
import sklearn.metrics as skm
from tabulate import tabulate
os.chdir("/afs/cs.stanford.edu/u/awni/scr/ecg-master")
sys.path.append('./ecg')

model_path = "/deep/group/pranavsr/saved/default/1499315864-990/0.424-0.848-009-0.354-0.868.hdf5"

import load
import json
import util
import predict

In [2]:
params = util.get_model_params(model_path)
test_params = json.load(open('./configs/test.json', 'r'))
x_gt, gt, processor, dl = load.load_test(
            test_params,
            train_params=params,
            split='test')
probs = predict.get_model_pred_probs(model_path, x_gt)

0it [00:00, ?it/s]

Building blacklist...


336it [00:00, 690.03it/s]
0it [00:00, ?it/s]

Contains 328 patients


64121it [00:22, 2831.58it/s]
  0%|          | 98/56378 [00:00<00:57, 973.39it/s]

...

Length of training set 56378
Length of test set 6346
Loading using processor...


336it [00:00, 706.92it/s]
0it [00:00, ?it/s]
  4%|▍         | 14/336 [00:00<00:02, 136.28it/s]

Contains 328 patients
Constructing Training Set...
Constructing Test Set...


100%|██████████| 336/336 [00:02<00:00, 140.29it/s]
Using TensorFlow backend.


Relabelling Classes...
Applying Normalization...
Length of training set 0
Length of test set 336
Size: 336 examples.
336/336 [==============================] - 7s     


In [41]:
import human_performance as hp
h_gt, h_probs = hp.human_gt_and_probs(test_params, x_gt, gt, processor)

0it [00:00, ?it/s]

Loading using processor...


336it [00:00, 984.38it/s]
0it [00:00, ?it/s]
 46%|████▌     | 155/336 [00:00<00:00, 1546.66it/s]

Contains 328 patients
Constructing Training Set...
Constructing Test Set...


100%|██████████| 336/336 [00:00<00:00, 1544.29it/s]


...

Relabelling Classes...
Applying Normalization...
Length of training set 0
Length of test set 336
Size: 336 examples.


336it [00:00, 7469.50it/s]
0it [00:00, ?it/s]
 39%|███▊      | 130/336 [00:00<00:00, 1291.40it/s]

Loading using processor...
Contains 328 patients
Constructing Training Set...
Constructing Test Set...


100%|██████████| 336/336 [00:00<00:00, 1240.62it/s]


Relabelling Classes...
Applying Normalization...
Length of training set 0
Length of test set 336
Size: 336 examples.


In [81]:
def merge(c1, c2, probs, gt):
    print probs.shape
    print gt.shape

def seq_auc(gt, probs, index):
    n_gts = np.zeros_like(gt)
    n_gts[gt==index] = 1
    n_ps = probs[:,:,index].squeeze()
    n_gts, n_ps = n_gts.ravel(), n_ps.ravel()
    return skm.roc_curve(n_gts, n_ps, pos_label=1), skm.roc_auc_score(n_gts, n_ps)

def set_auc(gt, probs, index):
    n_classes = np.max(gt)
    gt = gt.squeeze()
    n_gts = np.zeros_like(gt)
    n_gts[gt==index] = 1
    n_gts = np.max(n_gts, axis=1)
    n_ps = np.max(probs[:,:,index].squeeze(), axis=1)
    n_gts, n_ps = n_gts.ravel(), n_ps.ravel()
    return skm.roc_curve(n_gts, n_ps, pos_label=1), skm.roc_auc_score(n_gts, n_ps)

def print_scores(h_gt, h_probs, classes):
    h_ps = np.argmax(h_probs, axis=2)
    c_report = skm.classification_report(h_gt.ravel(), h_ps.ravel(), target_names=classes, digits=3)
    print c_report
    cnf = skm.confusion_matrix(h_gt.ravel(), h_ps.ravel())
    cat_classes = [c[:4] for c in classes]
    t_cnf = np.hstack([np.array(cat_classes)[:,None], cnf])
    print tabulate(t_cnf, headers=[''] + cat_classes)
    
def merge(c1, c2, h_gt, probs):
        
    classes = list(processor.classes)
    classes.remove(c2)
    
    class_to_int = {v:k for k,v in processor.int_to_class.items()}
    c1 = class_to_int[c1]
    c2 = class_to_int[c2]
    nc1 = min(c1, c2)
    nc2 = max(c1, c2)
    
    h_gt = np.copy(h_gt)
    h_gt[h_gt == nc2] = nc1
    h_gt[h_gt >= nc2] -= 1 
    
    shape = list(probs.shape)
    shape[-1] -= 1
    n_probs = np.zeros(shape)
    n_probs[..., :nc2] = probs[..., :nc2]
    n_probs[..., nc2:] = probs[..., nc2+1:]
    n_probs[..., nc1] += probs[..., nc2]

    return h_gt, n_probs, classes


# Model to Group comparisons

### AUC Results for Sequence and Set

In [39]:
print "Rhythm      \tSeq AUC\tSet AUC"
for idx, cname in processor.int_to_class.items():
    sq = seq_auc(gt, probs, idx)[1]
    st = set_auc(gt, probs, idx)[1]
    print "{: <8}\t{:.3f}\t{:.3f}".format(cname, sq, st)

Rhythm      	Seq AUC	Set AUC
AFIB    	0.947	0.940
AFL     	0.989	0.984
AVB_TYPE2	0.978	0.967
BIGEMINY	0.998	0.994
CHB     	0.988	0.978
EAR     	0.908	0.934
IVR     	0.995	0.986
JUNCTIONAL	0.987	0.980
NOISE   	0.981	0.949
SINUS   	0.973	0.982
SVT     	0.965	0.930
TRIGEMINY	0.997	0.996
VT      	0.995	0.981
WENCKEBACH	0.978	0.980


### Unmerged

In [78]:
print_scores(gt, probs, processor.classes)

             precision    recall  f1-score   support

       AFIB      0.838     0.458     0.592       642
        AFL      0.536     0.973     0.691       369
  AVB_TYPE2      0.684     0.679     0.681       364
   BIGEMINY      0.905     0.884     0.895       216
        CHB      0.814     0.846     0.830       487
        EAR      0.613     0.457     0.523       256
        IVR      0.836     0.626     0.716       195
 JUNCTIONAL      0.873     0.530     0.659       570
      NOISE      0.919     0.747     0.824       605
      SINUS      0.821     0.941     0.877      3034
        SVT      0.597     0.380     0.464       308
  TRIGEMINY      0.866     0.956     0.909       251
         VT      0.366     0.872     0.516        47
 WENCKEBACH      0.701     0.714     0.707       384

avg / total      0.793     0.780     0.771      7728

        AFIB    AFL    AVB_    BIGE    CHB    EAR    IVR    JUNC    NOIS    SINU    SVT    TRIG    VT    WENC
----  ------  -----  ------  ------  --

### Merge AFIB and AFL (marked as AFIB)

In [82]:
print_scores(*merge("AFIB", "AFL", gt, probs))

             precision    recall  f1-score   support

       AFIB      0.799     0.819     0.809      1011
  AVB_TYPE2      0.684     0.679     0.681       364
   BIGEMINY      0.905     0.884     0.895       216
        CHB      0.814     0.846     0.830       487
        EAR      0.613     0.457     0.523       256
        IVR      0.836     0.626     0.716       195
 JUNCTIONAL      0.873     0.530     0.659       570
      NOISE      0.922     0.747     0.826       605
      SINUS      0.823     0.941     0.878      3034
        SVT      0.589     0.364     0.450       308
  TRIGEMINY      0.866     0.956     0.909       251
         VT      0.366     0.872     0.516        47
 WENCKEBACH      0.701     0.714     0.707       384

avg / total      0.803     0.801     0.794      7728

        AFIB    AVB_    BIGE    CHB    EAR    IVR    JUNC    NOIS    SINU    SVT    TRIG    VT    WENC
----  ------  ------  ------  -----  -----  -----  ------  ------  ------  -----  ------  ----  ---

### Merge CHB and AVB_TYPE2 (marked as CHB)

In [84]:
print_scores(*merge("CHB", "AVB_TYPE2", gt, probs))

             precision    recall  f1-score   support

       AFIB      0.838     0.458     0.592       642
        AFL      0.536     0.973     0.691       369
   BIGEMINY      0.843     0.864     0.853       851
        CHB      0.905     0.884     0.895       216
        EAR      0.613     0.457     0.523       256
        IVR      0.836     0.626     0.716       195
 JUNCTIONAL      0.873     0.530     0.659       570
      NOISE      0.919     0.747     0.824       605
      SINUS      0.822     0.941     0.878      3034
        SVT      0.597     0.380     0.464       308
  TRIGEMINY      0.866     0.956     0.909       251
         VT      0.366     0.872     0.516        47
 WENCKEBACH      0.701     0.714     0.707       384

avg / total      0.803     0.789     0.781      7728

        AFIB    AFL    BIGE    CHB    EAR    IVR    JUNC    NOIS    SINU    SVT    TRIG    VT    WENC
----  ------  -----  ------  -----  -----  -----  ------  ------  ------  -----  ------  ----  -----

# Individual to Group comparisons
### Unmerged

In [68]:
print_scores(h_gt, h_probs, processor.classes)

             precision    recall  f1-score   support

       AFIB      0.495     0.537     0.515      3852
        AFL      0.600     0.674     0.635      2214
  AVB_TYPE2      0.519     0.556     0.537      2184
   BIGEMINY      0.845     0.829     0.837      1296
        CHB      0.806     0.620     0.701      2922
        EAR      0.636     0.380     0.476      1536
        IVR      0.654     0.611     0.632      1170
 JUNCTIONAL      0.757     0.624     0.684      3420
      NOISE      0.788     0.749     0.768      3630
      SINUS      0.803     0.898     0.847     18204
        SVT      0.500     0.408     0.449      1848
  TRIGEMINY      0.859     0.828     0.843      1506
         VT      0.500     0.652     0.566       282
 WENCKEBACH      0.661     0.539     0.593      2304

avg / total      0.723     0.724     0.719     46368

        AFIB    AFL    AVB_    BIGE    CHB    EAR    IVR    JUNC    NOIS    SINU    SVT    TRIG    VT    WENC
----  ------  -----  ------  ------  --

### Merge AFIB and AFL (marked as AFIB)

In [75]:
print_scores(*merge("AFIB", "AFL", h_gt, h_probs))

             precision    recall  f1-score   support

       AFIB      0.648     0.712     0.679      6066
  AVB_TYPE2      0.519     0.556     0.537      2184
   BIGEMINY      0.845     0.829     0.837      1296
        CHB      0.806     0.620     0.701      2922
        EAR      0.636     0.380     0.476      1536
        IVR      0.654     0.611     0.632      1170
 JUNCTIONAL      0.757     0.624     0.684      3420
      NOISE      0.788     0.749     0.768      3630
      SINUS      0.803     0.898     0.847     18204
        SVT      0.500     0.408     0.449      1848
  TRIGEMINY      0.859     0.828     0.843      1506
         VT      0.500     0.652     0.566       282
 WENCKEBACH      0.661     0.539     0.593      2304

avg / total      0.738     0.741     0.735     46368

        AFIB    AVB_    BIGE    CHB    EAR    IVR    JUNC    NOIS    SINU    SVT    TRIG    VT    WENC
----  ------  ------  ------  -----  -----  -----  ------  ------  ------  -----  ------  ----  ---

### Merge CHB and AVB_TYPE2 (marked as CHB)

In [76]:
print_scores(*merge("CHB", "AVB_TYPE2", h_gt, h_probs))

             precision    recall  f1-score   support

       AFIB      0.495     0.537     0.515      3852
        AFL      0.600     0.674     0.635      2214
   BIGEMINY      0.813     0.731     0.770      5106
        CHB      0.845     0.829     0.837      1296
        EAR      0.636     0.380     0.476      1536
        IVR      0.654     0.611     0.632      1170
 JUNCTIONAL      0.757     0.624     0.684      3420
      NOISE      0.788     0.749     0.768      3630
      SINUS      0.803     0.898     0.847     18204
        SVT      0.500     0.408     0.449      1848
  TRIGEMINY      0.859     0.828     0.843      1506
         VT      0.500     0.652     0.566       282
 WENCKEBACH      0.661     0.539     0.593      2304

avg / total      0.737     0.739     0.735     46368

        AFIB    AFL    BIGE    CHB    EAR    IVR    JUNC    NOIS    SINU    SVT    TRIG    VT    WENC
----  ------  -----  ------  -----  -----  -----  ------  ------  ------  -----  ------  ----  -----